In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
#dataset 구축축
actions = [
    'red',
    'blue',
    'green',
    'light up',
    'light down'
]

data = np.concatenate([
    np.load(r'C:\Users\UESR\OneDrive\바탕 화면\CV\스마트무드등\data\dataset.csv\seq_red_1726364198.npy'),
    np.load(r'C:\Users\UESR\OneDrive\바탕 화면\CV\스마트무드등\data\dataset.csv\seq_green_1726364198.npy'),
    np.load(r'C:\Users\UESR\OneDrive\바탕 화면\CV\스마트무드등\data\dataset.csv\seq_blue_1726364198.npy'),
    np.load(r'C:\Users\UESR\OneDrive\바탕 화면\CV\스마트무드등\data\dataset.csv\seq_up_1726364198.npy'),
    np.load(r'C:\Users\UESR\OneDrive\바탕 화면\CV\스마트무드등\data\dataset.csv\seq_down_1726364198.npy'),
], axis=0)

data.shape

(4010, 30, 100)

In [3]:
#데이터 라벨 분리
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(4010, 30, 99)
(4010,)


In [4]:
#one-heat 인코딩
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(4010, 5)

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

# x_data와 y_data가 이미 정의되어 있다고 가정
x_data = x_data.astype(np.float32)  # 입력 데이터
y_data = y_data.astype(np.float32)  # 라벨 데이터

# 데이터셋을 9:1로 분할 (train set: 90%, test set: 10%)
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

# 데이터셋 크기 출력
print(f"학습 데이터 크기: {x_train.shape}, 라벨 데이터 크기: {y_train.shape}")
print(f"검증 데이터 크기: {x_val.shape}, 라벨 데이터 크기: {y_val.shape}")


학습 데이터 크기: (3609, 30, 99), 라벨 데이터 크기: (3609, 5)
검증 데이터 크기: (401, 30, 99), 라벨 데이터 크기: (401, 5)


In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Self-Attention
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x + inputs)
    
    # Feed Forward Network
    x_ff = Dense(ff_dim, activation='relu')(x)
    x_ff = Dense(inputs.shape[-1])(x_ff)
    x_ff = Dropout(dropout)(x_ff)
    x = LayerNormalization(epsilon=1e-6)(x + x_ff)
    return x


# 입력 층 정의
input_shape = x_train.shape[1:]  # (시퀀스 길이, 특징 차원)
inputs = Input(shape=input_shape)

# 트랜스포머 인코더 블록 적용
x = transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=64, dropout=0.1)

# 필요하면 트랜스포머 블록을 더 추가
# x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=64, dropout=0.1)

# 출력층 구성
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(len(actions), activation='softmax')(x)

# 모델 생성
model = Model(inputs, outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# 모델 요약 정보 출력
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 99)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 30, 99)    │    102,243 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 30, 99)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 30, 99)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 30, 99)    │        198 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 30, 64)    │      6,400 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 30, 99)    │      6,435 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 30, 99)    │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 30, 99)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 99)    │        198 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 99)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 99)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      3,200 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 32)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5)         │        165 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 118,839 (464.21 KB)

 Trainable params: 118,839 (464.21 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
#모델 학습

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,  # 에포크 수는 데이터에 따라 조정
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)


Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - acc: 0.8577 - loss: 0.4197 - val_acc: 1.0000 - val_loss: 5.6775e-04
Epoch 2/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - acc: 0.9988 - loss: 0.0082 - val_acc: 1.0000 - val_loss: 0.0026
Epoch 3/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - acc: 0.9997 - loss: 0.0047 - val_acc: 1.0000 - val_loss: 1.5275e-04
Epoch 4/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - acc: 1.0000 - loss: 0.0010 - val_acc: 1.0000 - val_loss: 5.4295e-05
Epoch 5/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - acc: 1.0000 - loss: 5.8933e-04 - val_acc: 1.0000 - val_loss: 2.2878e-05
Epoch 6/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - acc: 1.0000 - loss: 9.5863e-04 - val_acc: 1.0000 - val_loss: 1.4983e-05
Epoch 7/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - acc: 1.0000 - loss: 2.8478e-04 - val_acc: 1.0000 - val_loss: 8.1775e-06
Epoch 8/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - acc: 1.0000 - loss: 2.7698e-04 - val_acc: 1.0000 - val_loss: 5.8293e-06
Epoc

In [13]:
# 검증 데이터로 모델 평가
loss, accuracy = model.evaluate(x_val, y_val)
print(f'검증 손실: {loss}, 검증 정확도: {accuracy}')

# 새로운 데이터로 예측
predictions = model.predict(x_val)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_val, axis=1)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 1.0000 - loss: 3.1385e-10
검증 손실: 2.972800061673553e-10, 검증 정확도: 1.0
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


In [25]:
model.save('transformer_model.keras')  # Keras 기본 형식으로 저장
